In [17]:
import os
import random
import shutil
from torchvision import datasets
from sklearn.model_selection import train_test_split
from collections import defaultdict
from PIL import Image

# Load CIFAR-100 dataset
cifar100 = datasets.CIFAR100(root='./data', download=True)

# Function to create directories if they don't exist
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Base path for the organized data
base_dir = './cifar100_data_split'

# Superclass to classes mapping (manually mapping specific classes for each superclass)
superclass_to_classes = {
    "aquatic_mammals": ["beaver", "dolphin", "otter", "seal", "whale"],
    "fish": ["aquarium_fish", "flatfish", "ray", "shark", "trout"],
    "food containers": ["bottle", "bowl", "can", "cup", "plate"],
    "fruit and vegetables": ["apple", "orange", "mushrooms", "pear", "sweet_peppers"],
    "household electrical devices": ["clock", "keyboard", "lamp", "telephone", "television"],
    "household furniture": ["bed", "chair", "couch", "table", "wardrobe"],
    "insects": ["bee", "beetle", "butterfly", "caterpillar", "cockroach"],
    "large carnivores": ["bear", "leopard", "lion", "tiger", "wolf"],
    "large man-made outdoor things": ["bridge", "castle", "house", "road", "skyscraper"],
    "large natural outdoor scenes": ["cloud", "forest", "mountain", "plain", "sea"],
    "large omnivores and herbivores": ["camel", "cattle", "chimpanzee", "elephant", "kangaroo"],
    "medium-sized mammals": ["fox", "porcupine", "possum", "raccoon", "skunk"],
    "non-insect invertebrates": ["crab", "lobster", "snail", "spider", "worm"],
    "people": ["baby", "boy", "girl", "man", "woman"],
    "reptiles": ["crocodile", "lizard", "snake", "turtle", "dinosaur"],
    "small mammals": ["hamster", "mouse", "rabbit", "shrew", "squirrel"],
    "trees": ["maple_tree", "oak_tree", "palm_tree", "pine_tree", "willow_tree"],
    "vehicles 1": ["bicycle", "bus", "motorcycle", "pickup_truck", "train"],
    "vehicles 2": ["lawn_mower", "rocket", "streetcar", "tank", "tractor"]
}

# Function to ensure that we have equal number of images for each class under each superclass
def get_balanced_data_for_superclass():
    data_by_class = defaultdict(list)
    
    # Collect the indices of the images per class
    for idx, label in enumerate(cifar100.targets):
        class_name = cifar100.classes[label]
        # Only collect data for classes that belong to the current superclass
        for superclass, class_list in superclass_to_classes.items():
            if class_name in class_list:
                data_by_class[class_name].append(idx)
    
    # Ensure equal number of images by finding the minimum number of images in any class
    min_images_per_class = min(len(images) for images in data_by_class.values())
    print(f"Minimum number of images per class: {min_images_per_class}")
    
    # Randomly sample images to ensure equal class distribution
    balanced_indices = {}
    for class_name, indices in data_by_class.items():
        balanced_indices[class_name] = random.sample(indices, min_images_per_class)
    
    return balanced_indices

# Split data into train, val, and test sets
def split_data_into_train_val_test(balanced_indices):
    train_data = []
    val_data = []
    test_data = []
    
    # Split data into train, val, test for each class
    for class_name, indices in balanced_indices.items():
        train, temp = train_test_split(indices, test_size=0.5, random_state=42)
        val, test = train_test_split(temp, test_size=0.5, random_state=42)
        
        train_data.extend(train)
        val_data.extend(val)
        test_data.extend(test)
    
    return train_data, val_data, test_data

# Function to save images into the correct directory structure
def save_images(data, split_name):
    for idx in data:
        image, label = cifar100[idx]
        class_name = cifar100.classes[label]
        
        # Find the superclass based on class_name
        for superclass, class_list in superclass_to_classes.items():
            if class_name in class_list:
                break
        
        # Create destination path
        dest_dir = os.path.join(base_dir, split_name, superclass, class_name)
        create_dir(dest_dir)  # Make sure the destination directory exists
        
        # Save the image to the destination folder
        image_name = f"{idx}.png"  # Use the index as the filename
        image.save(os.path.join(dest_dir, image_name))

# Main function to create directories, balance data, and save images
def main():
    # Create base directories
    for split in ['train', 'val', 'test']:
        for superclass in superclass_to_classes:
            for class_name in superclass_to_classes[superclass]:
                class_path = os.path.join(base_dir, split, superclass, class_name)
                create_dir(class_path)
    
    # Get balanced data for each superclass
    balanced_indices = get_balanced_data_for_superclass()
    
    # Split data into train, val, and test sets
    train_data, val_data, test_data = split_data_into_train_val_test(balanced_indices)
    
    # Save the images in their respective directories
    save_images(train_data, 'train')
    save_images(val_data, 'val')
    save_images(test_data, 'test')
    
    print("CIFAR-100 data successfully divided into train, val, and test splits with equal class distribution.")

# Run the process
if __name__ == "__main__":
    main()


Files already downloaded and verified
Minimum number of images per class: 500
CIFAR-100 data successfully divided into train, val, and test splits with equal class distribution.


In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [19]:
# check if GPU is available
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"using device: {device}")

using device: cuda


In [20]:
# Dataset Paths
train_dir=r'C:\Users\user\Desktop\Deep Learning\CNN-CIFAR-100\cifar100_data_split\test'
test_dir=r'C:\Users\user\Desktop\Deep Learning\CNN-CIFAR-100\cifar100_data_split\train'
val_dir=r'C:\Users\user\Desktop\Deep Learning\CNN-CIFAR-100\cifar100_data_split\val'

In [21]:
# Check Directory and Files Manually
import os

print("Train directory structure:")
for superclass in os.listdir(train_dir):
    print(superclass, os.listdir(os.path.join(train_dir, superclass)))


Train directory structure:
aquatic_mammals ['beaver', 'dolphin', 'otter', 'seal', 'whale']
fish ['aquarium_fish', 'flatfish', 'ray', 'shark', 'trout']
food containers ['bottle', 'bowl', 'can', 'cup', 'plate']
fruit and vegetables ['apple', 'mushrooms', 'orange', 'pear', 'sweet_peppers']
household electrical devices ['clock', 'keyboard', 'lamp', 'telephone', 'television']
household furniture ['bed', 'chair', 'couch', 'table', 'wardrobe']
insects ['bee', 'beetle', 'butterfly', 'caterpillar', 'cockroach']
large carnivores ['bear', 'leopard', 'lion', 'tiger', 'wolf']
large man-made outdoor things ['bridge', 'castle', 'house', 'road', 'skyscraper']
large natural outdoor scenes ['cloud', 'forest', 'mountain', 'plain', 'sea']
large omnivores and herbivores ['camel', 'cattle', 'chimpanzee', 'elephant', 'kangaroo']
medium-sized mammals ['fox', 'porcupine', 'possum', 'raccoon', 'skunk']
non-insect invertebrates ['crab', 'lobster', 'snail', 'spider', 'worm']
people ['baby', 'boy', 'girl', 'man', 

In [22]:
# Data normalization and augmentation
from torchvision import transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Example normalization
])


In [23]:
# Load Datasets
train_dataset=datasets.ImageFolder(train_dir, transform=transform)
test_dataset=datasets.ImageFolder(test_dir, transform=transform)
val_dataset=datasets.ImageFolder(val_dir, transform=transform)

# Create data loaders
train_loader=DataLoader(train_dataset,batch_size=64,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=64,shuffle=False)
val_loader=DataLoader(val_dataset,batch_size=64,shuffle=False)

In [30]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1) # Conv Layer 1
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0) # Pooling Loyer
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1) # Conv Layer 2
        self.fc1 = nn.Linear(64*8*8, 128) # Fully connected Layer
        self.fc2 = nn.Linear(128, 95)
        
# Output layer for 10 classes
    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 64*8*8) #Flatten
        x = nn.functional.relu(self.fc1(x)) # Fully connected Layer
        x = self.fc2(x) # Output Layer
        return x

# Initialize the model, loss function, and optimizer
model=CNN().to(device)    # move model to GPU
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters())

In [31]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        # move data to GPU
        images=images.to(device)
        labels=labels.to(device)
        
        optimizer.zero_grad()
        outputs =model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
    print (f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

Epoch [1/10], Loss: 2.5671
Epoch [2/10], Loss: 2.1455
Epoch [3/10], Loss: 1.9230
Epoch [4/10], Loss: 1.7339
Epoch [5/10], Loss: 1.5384
Epoch [6/10], Loss: 1.3521
Epoch [7/10], Loss: 1.1618
Epoch [8/10], Loss: 0.9798
Epoch [9/10], Loss: 0.7810
Epoch [10/10], Loss: 0.6023


In [32]:
# Evaluate the model on test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        # Move data to GPU
        images=images.to(device)
        labels=labels.to(device)
        
        outputs = model(images)
        _, predicted = torch. max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy: 2f}%")

Test Accuracy:  40.292473%


In [33]:
# Evaluate the model on test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        # Move data to GPU
        images=images.to(device)
        labels=labels.to(device)
        
        outputs = model(images)
        _, predicted = torch. max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
val_accuracy = 100 * correct / total
print(f"Validation Accuracy: {val_accuracy: 2f}%")

Validation Accuracy:  39.698925%
